In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import intake

import pandas as pd

import dask
from dask.distributed import Client

from fluxnet_etl import load_x_df, load_y_df, make_cyclic_doy
from utils import get_training_sites, extract_site_meta

from metsim.methods import mtclim

In [3]:
!mkdir -p metsim_data

In [4]:
cat = intake.Catalog('./fluxnet/catalog.yml')

glob_path = './fluxnet/FLX_*_FLUXNET2015_FULLSET_DD_*.csv'
thresh_days = 365
train_sites = get_training_sites(glob_path, thresh_days)
train_sites.describe()

,dur
count,200
mean,2774 days 22:26:24
std,1823 days 21:04:02.933819
min,729 days 00:00:00
25%,1095 days 00:00:00
50%,1826 days 00:00:00
75%,4017 days 00:00:00
max,8035 days 00:00:00


In [5]:
all_site_meta = pd.read_excel('./fluxnet/FLX_AA-Flx_BIF_LATEST.xlsx').set_index(['SITE_ID', 'VARIABLE'])['DATAVALUE']
meta = {key: extract_site_meta(all_site_meta, key) for key in train_sites.index}

In [6]:
from metsim.metsim import MetSim, wrap_run_cell
import metsim.constants as cnst

param_keys = ['sw_prec_thresh', 'lw_cloud', 'lw_type', 'tday_coef', 'tdew_tol', 'tmax_daylength_fraction', 'rain_scalar', 'lapse_rate']
params = {key: MetSim.params[key] for key in param_keys}
params

{'sw_prec_thresh': 0.0,
 'lw_cloud': 'cloud_deardorff',
 'lw_type': 'prata',
 'tday_coef': 0.45,
 'tdew_tol': 1e-06,
 'tmax_daylength_fraction': 0.67,
 'rain_scalar': 0.75,
 'lapse_rate': 0.0065}

In [7]:
client = Client(n_workers=12)
client

/glade/u/home/jhamman/projects/envs/met-ml/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:41133 Dashboard: proxy/41789/status,Cluster Workers: 12 Cores: 12 Memory: 42.95 GB


In [8]:
@dask.delayed
def run_metsim(df, site_meta):

    df['dtr'] = df['t_max'] - df['t_min']
    df['smoothed_dtr'] = df['dtr'].rolling(30).mean()
    df['seasonal_prec'] = cnst.DAYS_PER_YEAR * df['P'].rolling(90).mean()
    df['lat'] = site_meta['lat']
    df['lon'] = site_meta['lon']
    df['elev'] = site_meta['elev']
    df.index.name = 'time'

    ds = df.rename(columns={'P': 'prec'}).to_xarray().isel(time=slice(89, None))

    state = None
    disagg = False
    out_times = ds.indexes['time']
    return wrap_run_cell(mtclim.run, params, ds, state, disagg, out_times)[0]


@dask.delayed
def write_metsim(df, fpath):
    df.to_csv(fpath)
    return fpath

In [9]:
tasks = []
for name, site_meta in meta.items():
    print(name)
    df = load_x_df(cat['subdaily'](site=name), site_meta)
    out = run_metsim(df, site_meta)
    tasks.append(write_metsim(out, f'./metsim_data/metsim_{name}_DD.csv'))

IT-SRo
CA-Qfo
US-Me3
DK-Sor
AR-Vir
AU-Wac
CA-Oas
CA-NS3
CA-NS7
AR-SLu
IT-Col
US-NR1
RU-Fyo
AU-Rig
CA-TP4
CA-TP3
US-Goo
ZA-Kru
ES-LgS
US-Var
CA-Gro
AU-Stp
ES-LJu
US-Ne3
IT-Noe
US-ARb
IT-Ro1
US-Wi3
US-Tw4
CH-Oe2
PA-SPs
CA-NS4
DE-RuS
CN-Du2
US-Syv
US-UMB
US-GBT
US-Ivo
CA-TP2
DE-Akm
US-Me6
DE-RuR
JP-SMF
FR-Pue
DE-Hai
US-Lin
CA-TP1
US-Tw2
CH-Lae
CH-Fru
AU-How
SE-St1
AU-DaP
DK-NuF
CA-SF1
IT-CA1
US-Wi4
CN-Sw2
AU-Dry
BE-Vie
CG-Tch
US-WPT
AU-Emr
CN-Din
US-Ton
US-AR2
DK-Eng
IT-MBo
AU-DaS
NL-Hor
US-Wi9
US-Tw1
CA-NS5
IT-Tor
US-Ne1
CN-Cng
NO-Adv
AU-Whr
US-ARM
US-Tw3
US-GLE
RU-Ha1
US-KS2
CA-Obs
AU-Fog
US-IB2
US-Twt
IT-Ro2
AU-RDF
IT-BCi
SN-Dhr
US-LWW
IT-CA2
IT-Ren
US-Ne2
DE-Lkb
GF-Guy
RU-Sam
IT-CA3
AU-Cum
US-Myb
IT-SR2
CA-Man
CN-Qia
IT-Cp2
FI-Lom
US-Blo
CN-Du3
IT-Isp
DE-Kli
CN-Ha2
RU-Cok
FI-Jok
ZM-Mon
FI-Sod
CN-Cha
DE-Geb
CA-NS1
DE-Lnf
US-SRG
US-Oho
DE-Zrk
NL-Loo
AU-Gin
US-Cop
US-AR1
CH-Cha
US-Me5
US-CRT
US-Me2
US-Sta
BE-Lon
BR-Sa1
US-MMS
DE-Obe
BR-Sa3
AU-Ade
US-Wi6
AU-Wom
FI-Hyy
CN-HaM
US-Los
DE-Gri

In [10]:
dask.persist(tasks)

([Delayed('write_metsim-997fbfe2-1b5f-4e78-95b1-24e57e9d83b3'),
  Delayed('write_metsim-45047546-6eea-41a0-801d-866dd7e78fc1'),
  Delayed('write_metsim-056c726c-1205-401f-a439-794aba7c9548'),
  Delayed('write_metsim-c60bc160-71a0-4e75-a58c-dc077fe6a2f2'),
  Delayed('write_metsim-04241fae-48a8-4e67-ad2c-d060c82ace1f'),
  Delayed('write_metsim-3ea9661c-ea4f-44f3-9bcb-6e1a4caa553b'),
  Delayed('write_metsim-52e9c6bd-0513-492c-96a9-d4be6fe685a4'),
  Delayed('write_metsim-a969d51f-7d7e-4aab-9b98-3724959798d6'),
  Delayed('write_metsim-98afd1ec-ad01-482a-95fc-2e2eb6266c6d'),
  Delayed('write_metsim-50e0a690-e089-4cd4-8f5d-36c432ba0df1'),
  Delayed('write_metsim-72e7e28a-a09e-4f1c-84e7-848b7ae6e2b9'),
  Delayed('write_metsim-30a1f930-b61a-4072-a919-4ccf37253ea7'),
  Delayed('write_metsim-9bb6c23d-8836-4e30-a5e3-5d681f6bea38'),
  Delayed('write_metsim-8b8347fc-4fbc-4b20-ba15-914a8df55a27'),
  Delayed('write_metsim-6e681102-8d11-4b04-8f50-052d82202449'),
  Delayed('write_metsim-5ccce17f-407f-44